In [90]:
import numpy as np
import pandas as pd

datapath = "Z:/hcs/data/ipeds/"

In [113]:
# read in raw data
df = pd.read_stata(datapath + '/ipeds_c_all.dta')

In [114]:
# remove rows with missing cipcode in 2010
df = df[df['cipcode2010'] != '']

# keep only bachelor's degrees
df = df[df['awlevel'] == 5]

# drop aggregate values
df = df[(df['cipcode'] != '99') & (df['cipcode'] != '99.') &
        (df['cipcode'] != '99.0000') & (df['cipcode'] != '95.0000') &
        (df['cipcode'] != '95.9500')]

# check if there are duplicated values according to original cip code
# note that these may be aggregated
assert False == df.duplicated(subset=
                              ['year', 'unitid', 'cipcode', 'majornum']).any()

# create two digit and 4 digit cipcodes
df['cip2'] = df['cipcode2010'].str[:2]
df['cip4'] = df['cipcode2010'].str[:5]

In [119]:
# aggregate over cipcode2010, majornum
df.groupby(['year','unitid','cipcode2010','cip4','cip2']).aggregate('sum')

df.set_index(['year','unitid','cipcode2010'])['cipcode2010']

KeyError: 'cipcode2010'

In [112]:
# drop unnecessary columns
df = df.drop(['cipcode', 'awlevel'], axis=1)

In [109]:




df.head()

,year,unitid,cipcode2010,majornum,ctotalm,ctotalw,cip2,cip4
1141,2003.0,100654,01.0000,1.0,5.0,3.0,01,01.00
1142,2004.0,100654,01.0000,1.0,3.0,1.0,01,01.00
1143,2005.0,100654,01.0000,1.0,2.0,0.0,01,01.00
1144,2006.0,100654,01.0000,1.0,8.0,1.0,01,01.00
1145,2007.0,100654,01.0000,1.0,2.0,1.0,01,01.00
